# Extraction of robot parameters for multibody kinematics/ dynamics

In [1]:
import xml.etree.ElementTree as ET
from urdf_kit.graph.tree import kinematic_tree

In [2]:
# assuming current directory as the folder containing this file
import os
iiwa_urdf_fpath = os.path.join("..","tests","data","kuka_iiwa","model.urdf")
urdf_root = ET.parse(iiwa_urdf_fpath).getroot()

## `kinematic_tree`

A high-level API within `urdf_kit` for ...

* inspecting/ modifying the kinematic tree topology
* extracting (constant) multibody parameters relevant for Jacobians, dynamics computations,...

In [3]:
iiwa_mdl = kinematic_tree(urdf_root)

In [4]:
iiwa_mdl.print_graph(show_SE3=True)

------------------------------
robot name:  lbr_iiwa
root link name:  lbr_iiwa_link_0
 connections: (sorting by  depth_first )
 (format:  parent ---> [ joint ] ---> child link )
   lbr_iiwa_link_0 --[lbr_iiwa_joint_1]--> lbr_iiwa_link_1
   X_ParentJoint =
   1         0         0         0         
   0         1         0         0         
   0         0         1         0.1575    
   0         0         0         1         

   lbr_iiwa_link_1 --[lbr_iiwa_joint_2]--> lbr_iiwa_link_2
   X_ParentJoint =
  -1         0        -2.068e-13  0         
  -2.068e-13 -4.897e-12  1         0         
   0         1         4.897e-12  0.2025    
   0         0         0         1         

   lbr_iiwa_link_2 --[lbr_iiwa_joint_3]--> lbr_iiwa_link_3
   X_ParentJoint =
  -1         0        -2.068e-13  0         
  -2.068e-13 -4.897e-12  1         0.2045    
   0         1         4.897e-12  0         
   0         0         0         1         

   lbr_iiwa_link_3 --[lbr_iiwa_joint_4]--> lbr_ii

## Parameter extraction

### Assumptions for the parameters below:
the screw axis and home pose are  
* the ones **of** the child body (the second subscript, *B*)
* **relative to** the parent body (the first subscript, *A*)
* **expressed in** the parent body frame (the superscript, *C*)

### The "body frames" in questions mean different things.
* For kinematics parameters, it is the respective URDF link frames, implied by the respective XML element `<robot/link>`.
* For dynamics parameters, it is the respective Center of Mass (CoM)-aligned frames, contained in the respective XML element `<robot/link/inertial/origin>`

### spatial math convention:
* linear part-first 
  $$
  \boldsymbol{\hat \omega}_{A,B}^{(C)} = \begin{bmatrix} v_x \\ v_y \\ v_z \\ \omega_x \\ \omega_y \\ \omega_z \end{bmatrix}
  $$

* In the implementation, we will have something like `{physicalQuatity}_AB_C`, 

  In the example above, `physicalQuatity` is `screwAx`, which is a *normalized* twist 
  
  > Caution: it is not an ordinary Euclidean norm on that 6-vector, consult [2] for the meaning of a twist being normalized)

* Useful references:
  * [1] https://manipulation.csail.mit.edu/pick.html#monogram
  * [2] http://hades.mech.northwestern.edu/index.php/Modern_Robotics
  

In [5]:
# probably first fix some "continuous" joints (not applicable in this example)
iiwa_mdl.merge_fixed_joints() # recommended to do this first, although it's not required in this example

Attempting to remove fixed joints as much as possible
no fixed joints to be removed!


In [6]:
iiwa_mdl_dyn = iiwa_mdl.extract_dynamics(base_is_mobile=False)

In [7]:
iiwa_mdl_dyn.base_is_mobile

False

Below is the serialization format, see `urdf_kit::graph::params` for more details

In [8]:
print(iiwa_mdl_dyn)

robot_name: lbr_iiwa
base_mass: null
base_inertia: null
joints:
- joint_name: lbr_iiwa_joint_1
  home_pose:
  - [1.0, 0.0, 0.0, 0.1]
  - [0.0, 1.0, 0.0, -0.03]
  - [0.0, 0.0, 1.0, 0.2075]
  - [0.0, 0.0, 0.0, 1.0]
  screw_axis: [0.0, -0.1, 0.0, 0.0, 0.0, 1.0]
  parent_link_name: lbr_iiwa_link_0
  child_link_name: lbr_iiwa_link_1
  mass: 4.0
  inertia:
  - [0.1, 0.0, 0.0]
  - [0.0, 0.09, 0.0]
  - [0.0, 0.0, 0.02]
- joint_name: lbr_iiwa_joint_2
  home_pose:
  - [-1.0, 0.0, 0.0, -0.00030000000000868655]
  - [0.0, 0.0, 1.0, 0.07199999999971105]
  - [0.0, 1.0, 0.0, 0.14150000000020568]
  - [0.0, 0.0, 0.0, 1.0]
  screw_axis: [-0.08249999999985312, 0.0, 0.0, 0.0, 1.0, 0.0]
  parent_link_name: lbr_iiwa_link_1
  child_link_name: lbr_iiwa_link_2
  mass: 4.0
  inertia:
  - [0.05, 0.0, 0.0]
  - [0.0, 0.018, 0.0]
  - [0.0, 0.0, 0.044]
- joint_name: lbr_iiwa_joint_3
  home_pose:
  - [-1.0, 0.0, 0.0, -0.000300000000026887]
  - [0.0, 0.0, 1.0, 0.2754999999998531]
  - [0.0, 1.0, 0.0, -0.0119999999993634